In [1]:
import cv2
import os
import numpy as np
from PIL import Image
import tkinter as tk
from tkinter import messagebox

# Create the main window
window = tk.Tk()
window.title("Face Recognition System")

# Add Name, Age, and Address labels and input fields
l1 = tk.Label(window, text="Name", font=("Algerian", 20))
l1.grid(column=0, row=0)
t1 = tk.Entry(window, width=50, bd=5)
t1.grid(column=1, row=0)

l2 = tk.Label(window, text="Age", font=("Algerian", 20))
l2.grid(column=0, row=1)
t2 = tk.Entry(window, width=50, bd=5)
t2.grid(column=1, row=1)

l3 = tk.Label(window, text="Address", font=("Algerian", 20))
l3.grid(column=0, row=2)
t3 = tk.Entry(window, width=50, bd=5)
t3.grid(column=1, row=2)

# Function to generate the dataset
def generate_dataset():
    if t1.get() == "" or t2.get() == "" or t3.get() == "":
        messagebox.showinfo('Result', 'Please provide complete details of the user')
    else:
        face_classifier = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")

        def face_cropped(img):
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            faces = face_classifier.detectMultiScale(gray, 1.3, 5)
            if len(faces) == 0:
                return None
            for (x, y, w, h) in faces:
                cropped_face = img[y:y + h, x:x + w]
            return cropped_face

        cap = cv2.VideoCapture(0)
        id = 1  # User ID
        img_id = 0  # Image counter

        while True:
            ret, frame = cap.read()
            if not ret:
                print("Failed to capture image. Exiting...")
                break

            cropped_face = face_cropped(frame)
            if cropped_face is not None:
                img_id += 1
                face = cv2.resize(cropped_face, (200, 200))
                face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
                file_name_path = "data/user." + str(id) + "." + str(img_id) + ".jpg"
                cv2.imwrite(file_name_path, face)
                cv2.putText(face, str(img_id), (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 2)
                cv2.imshow("Cropped face", face)

            if cv2.waitKey(1) == 13 or img_id == 200:  # Break after Enter key or 200 images
                break

        cap.release()
        cv2.destroyAllWindows()
        print("Collecting samples is completed....")


# Button to generate dataset
b1 = tk.Button(window, text="Generate dataset", font=("Algerian", 20), bg="pink", fg="black", command=generate_dataset)
b1.grid(column=2, row=4)

# Function to train the classifier
def train_classifier():
    data_dir = "C:/Users/Divyanshu/OneDrive/Desktop/face recognizer/data"
    path = [os.path.join(data_dir, f) for f in os.listdir(data_dir)]

    faces = []
    ids = []

    for image in path:
        img = Image.open(image).convert('L')
        imageNp = np.array(img, 'uint8')
        id = int(os.path.split(image)[1].split(".")[1])
        faces.append(imageNp)
        ids.append(id)

    ids = np.array(ids)

    clf = cv2.face.LBPHFaceRecognizer_create()
    clf.train(faces, ids)
    clf.write("classifier.xml")
    messagebox.showinfo('Result', 'Training dataset completed')

# Button to train classifier
b2 = tk.Button(window, text="Training", font=("Algerian", 20), bg="orange", fg="red", command=train_classifier)
b2.grid(column=0, row=4)

# Function to detect faces
def detect_face():
    def draw_boundary(img, classifier, scaleFactor, minNeighbors, color, text, clf):
        gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        features = classifier.detectMultiScale(gray_img, scaleFactor, minNeighbors)

        for (x, y, w, h) in features:
            cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
            id, pred = clf.predict(gray_img[y:y + h, x:x + w])
            confidence = int(100 * (1 - pred / 300))

            if confidence > 70:
                if id == 1:
                    cv2.putText(img, "Vikas", (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 1, cv2.LINE_AA)
                elif id == 2:
                    cv2.putText(img, "Manish", (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 1, cv2.LINE_AA)
            else:
                cv2.putText(img, "UNKNOWN", (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 1, cv2.LINE_AA)

        return img

    faceCascade = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
    clf = cv2.face.LBPHFaceRecognizer_create()
    clf.read("classifier.xml")

    video_capture = cv2.VideoCapture(0)
    if not video_capture.isOpened():
        print("Error: Could not open video stream.")
        exit()

    while True:
        ret, img = video_capture.read()
        if not ret:
            print("Failed to capture image")
            break

        img = draw_boundary(img, faceCascade, 1.3, 6, (255, 255, 255), "Face", clf)
        cv2.imshow("Face Detection", img)

        if cv2.waitKey(1) & 0xFF == ord('q'):  # Close when 'q' is pressed
            break

    video_capture.release()
    cv2.destroyAllWindows()

# Button to detect faces
b3 = tk.Button(window, text="Detect the faces", font=("Algerian", 20), bg="green", fg="orange", command=detect_face)
b3.grid(column=1, row=4)

# Function to close the entire app when 'q' is pressed
def quit_app(event=None):
    window.quit()  # Close the Tkinter window

# Bind the 'q' key to quit the app
window.bind('<q>', quit_app)

# Set window size
window.geometry("800x200")
window.mainloop()
